## 1. Introduction — What is Information?

In everyday language, "information" means facts, data, or knowledge. But in mathematics and signal processing, **information** has a precise, quantifiable meaning: it represents the **reduction of uncertainty**.

This fundamental insight comes from **Claude Shannon's** groundbreaking 1948 paper "A Mathematical Theory of Communication", which established that:

1. **Information can be quantified** — measured in precise units called **bits**
2. **Information = Surprise** — the less expected an event, the more information it carries
3. **Uncertainty can be measured** — via a quantity called **entropy**

### Why Does This Matter for Neuroscience?

Information theory provides powerful tools for analyzing neural signals:

- **How much information** does a brain signal carry?
- **How much information is shared** between two signals? → This leads to connectivity measures!
- **How much information flows** from one signal to another? → This reveals causality!

In this notebook, we'll build the foundational concepts: **entropy** and **information content**. These are the building blocks for mutual information (D02) and transfer entropy (D03), which are powerful connectivity metrics.

> 💡 **Key insight**: Information = Surprise. The less expected something is, the more information it carries when it occurs.

## 2. Intuition — Surprise and Uncertainty

Before diving into mathematics, let's build intuition with thought experiments.

### Thought Experiment 1: Weather Forecasts

Consider the statement "Tomorrow will be sunny":

- **In the Sahara desert**: This is expected → carries **little information**
- **In London**: This is less expected → carries **more information**
- **"Tomorrow will snow" in the Sahara**: Extremely surprising → carries **a lot of information**!

The key principle: **Rare events carry more information than common events.**

### Thought Experiment 2: Coin Flips

- **Fair coin** (50% heads, 50% tails): The outcome is maximally uncertain → each flip provides **maximum information**
- **Biased coin** (99% heads, 1% tails): The outcome is predictable → each flip provides **little information** (usually just confirms what we expected)

### The Connection

- **Uncertainty** = how spread out the possibilities are
- **Information** = how much uncertainty is reduced by an observation

Now let's see how to **quantify** this precisely!

In [ ]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
from numpy.typing import NDArray
from typing import Tuple, Optional, Union, List
from scipy import stats
from scipy.signal import welch
import sys
sys.path.append("../../..")

from src.colors import COLORS
from src.plotting import configure_plots

configure_plots()

In [ ]:
# Visualization 1: Distributions with different uncertainty

fig, axes = plt.subplots(1, 3, figsize=(14, 4))

# Distribution 1: Uniform (maximum uncertainty)
probs_uniform = np.array([0.2, 0.2, 0.2, 0.2, 0.2])
axes[0].bar(range(5), probs_uniform, color=COLORS["signal_1"], edgecolor="white", linewidth=2)
axes[0].set_title("Uniform Distribution\n(Maximum Uncertainty)", fontsize=12, fontweight="bold")
axes[0].set_xlabel("Outcome")
axes[0].set_ylabel("Probability")
axes[0].set_ylim(0, 0.8)
axes[0].set_xticks(range(5))
axes[0].set_xticklabels(["A", "B", "C", "D", "E"])

# Distribution 2: Slightly peaked (moderate uncertainty)
probs_moderate = np.array([0.1, 0.15, 0.5, 0.15, 0.1])
axes[1].bar(range(5), probs_moderate, color=COLORS["signal_2"], edgecolor="white", linewidth=2)
axes[1].set_title("Peaked Distribution\n(Moderate Uncertainty)", fontsize=12, fontweight="bold")
axes[1].set_xlabel("Outcome")
axes[1].set_ylim(0, 0.8)
axes[1].set_xticks(range(5))
axes[1].set_xticklabels(["A", "B", "C", "D", "E"])

# Distribution 3: Highly peaked (low uncertainty)
probs_peaked = np.array([0.02, 0.03, 0.9, 0.03, 0.02])
axes[2].bar(range(5), probs_peaked, color=COLORS["signal_3"], edgecolor="white", linewidth=2)
axes[2].set_title("Highly Peaked Distribution\n(Low Uncertainty)", fontsize=12, fontweight="bold")
axes[2].set_xlabel("Outcome")
axes[2].set_ylim(0, 0.8)
axes[2].set_xticks(range(5))
axes[2].set_xticklabels(["A", "B", "C", "D", "E"])

plt.suptitle("Which distribution has the most uncertainty?", fontsize=14, fontweight="bold", y=1.02)
plt.tight_layout()
plt.show()

print("Answer: The UNIFORM distribution (left) has maximum uncertainty.")
print("When all outcomes are equally likely, we have no way to predict what will happen!")

## 3. Shannon Entropy — The Formula

**Shannon entropy** quantifies the uncertainty of a random variable. For a discrete random variable $X$ with possible outcomes $x_1, x_2, ..., x_n$ and probabilities $p(x_i)$:

$$H(X) = -\sum_{i=1}^{n} p(x_i) \log p(x_i)$$

### Understanding the Components

- **$p(x_i)$**: Probability of outcome $x_i$
- **$\log p(x_i)$**: Negative (since $p \leq 1$), so the negative sign makes $H$ positive
- **Sum**: Average over all possible outcomes, weighted by probability

### Units

- **Base 2** (log₂): Entropy in **bits** — most common in information theory
- **Base e** (ln): Entropy in **nats** — common in physics and machine learning
- Conversion: 1 nat ≈ 1.44 bits

### Convention

When $p = 0$, we define $0 \times \log(0) = 0$ (the limit as $p \to 0$).

### Interpretations

1. **Average surprise**: $H(X)$ = expected value of "surprise" $(-\log p)$
2. **Minimum encoding length**: Average bits needed to encode outcomes
3. **Uncertainty measure**: How unpredictable is $X$ before we observe it?

In [ ]:
# Visualization 2: Contribution of each probability to entropy

p_values = np.linspace(0.001, 0.999, 500)
contribution = -p_values * np.log2(p_values)

fig, ax = plt.subplots(figsize=(10, 5))

ax.plot(p_values, contribution, color=COLORS["signal_1"], linewidth=2.5)
ax.fill_between(p_values, contribution, alpha=0.3, color=COLORS["signal_1"])

# Mark maximum
p_max = 1 / np.e  # ≈ 0.368
contrib_max = -p_max * np.log2(p_max)
ax.axvline(p_max, color=COLORS["grid"], linestyle="--", linewidth=1.5, alpha=0.7)
ax.scatter([p_max], [contrib_max], color=COLORS["signal_2"], s=100, zorder=5)
ax.annotate(f"Maximum at p = 1/e ≈ {p_max:.3f}", xy=(p_max, contrib_max),
            xytext=(p_max + 0.15, contrib_max),
            fontsize=11, fontweight="bold",
            arrowprops=dict(arrowstyle="->", color="black"))

ax.set_xlabel("Probability p", fontsize=12)
ax.set_ylabel("-p × log₂(p)", fontsize=12)
ax.set_title("Contribution of Each Probability to Total Entropy", fontsize=14, fontweight="bold")
ax.set_xlim(0, 1)
ax.set_ylim(0, 0.55)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Each outcome contributes -p × log₂(p) to the total entropy.")
print("Very rare events (p≈0) and very common events (p≈1) contribute little.")
print("Events with intermediate probability contribute the most!")

## 4. Computing Entropy — Examples

Let's compute entropy for concrete examples to build understanding.

In [ ]:
def compute_entropy_discrete(
    probabilities: NDArray[np.float64],
    base: float = 2.0
) -> float:
    """
    Compute Shannon entropy of a discrete probability distribution.
    
    Parameters
    ----------
    probabilities : NDArray[np.float64]
        Probability distribution (must sum to 1).
    base : float, optional
        Logarithm base. Use 2 for bits, np.e for nats. Default is 2.
    
    Returns
    -------
    float
        Shannon entropy of the distribution.
    
    Examples
    --------
    >>> compute_entropy_discrete(np.array([0.5, 0.5]))
    1.0  # Fair coin = 1 bit
    """
    # Ensure probabilities sum to 1 (with tolerance)
    assert np.abs(np.sum(probabilities) - 1.0) < 1e-9, "Probabilities must sum to 1"
    
    # Filter out zeros to avoid log(0)
    p = probabilities[probabilities > 0]
    
    # Compute entropy
    if base == np.e:
        entropy = -np.sum(p * np.log(p))
    else:
        entropy = -np.sum(p * np.log(p) / np.log(base))
    
    return float(entropy)


def compute_entropy_from_counts(
    counts: NDArray[np.int64],
    base: float = 2.0
) -> float:
    """
    Estimate entropy from observed counts.
    
    Parameters
    ----------
    counts : NDArray[np.int64]
        Count of observations for each outcome.
    base : float, optional
        Logarithm base. Default is 2 (bits).
    
    Returns
    -------
    float
        Estimated Shannon entropy.
    """
    # Convert counts to probabilities
    total = np.sum(counts)
    if total == 0:
        return 0.0
    
    probabilities = counts / total
    return compute_entropy_discrete(probabilities, base)

In [ ]:
# Example 1: Fair coin
p_fair_coin = np.array([0.5, 0.5])
H_fair = compute_entropy_discrete(p_fair_coin)
print(f"Example 1: Fair coin")
print(f"  P(heads) = P(tails) = 0.5")
print(f"  H = {H_fair:.4f} bits")
print(f"  → Maximum entropy for 2 outcomes!\n")

# Example 2: Biased coin (90% heads)
p_biased = np.array([0.9, 0.1])
H_biased = compute_entropy_discrete(p_biased)
print(f"Example 2: Biased coin (90% heads)")
print(f"  P(heads) = 0.9, P(tails) = 0.1")
print(f"  H = {H_biased:.4f} bits")
print(f"  → Less entropy (more predictable)\n")

# Example 3: Fair die (6 sides)
p_die = np.ones(6) / 6
H_die = compute_entropy_discrete(p_die)
print(f"Example 3: Fair die (6 sides)")
print(f"  P(each side) = 1/6")
print(f"  H = {H_die:.4f} bits = log₂(6)")
print(f"  → Maximum entropy for 6 outcomes!\n")

# Example 4: Loaded die
p_loaded = np.array([0.4, 0.2, 0.15, 0.1, 0.1, 0.05])
H_loaded = compute_entropy_discrete(p_loaded)
print(f"Example 4: Loaded die")
print(f"  Non-uniform probabilities")
print(f"  H = {H_loaded:.4f} bits")
print(f"  → Less than fair die ({H_die:.4f} bits)")

In [ ]:
# Visualization 3: Comparing distributions and their entropy

examples = [
    ("Fair Coin", np.array([0.5, 0.5]), ["H", "T"]),
    ("Biased Coin\n(90% heads)", np.array([0.9, 0.1]), ["H", "T"]),
    ("Fair Die", np.ones(6)/6, ["1", "2", "3", "4", "5", "6"]),
    ("Loaded Die", np.array([0.4, 0.2, 0.15, 0.1, 0.1, 0.05]), ["1", "2", "3", "4", "5", "6"]),
]

fig, axes = plt.subplots(1, 4, figsize=(16, 4))
colors = [COLORS["signal_1"], COLORS["signal_2"], COLORS["signal_3"], COLORS["signal_4"]]

for ax, (title, probs, labels), color in zip(axes, examples, colors):
    H = compute_entropy_discrete(probs)
    ax.bar(range(len(probs)), probs, color=color, edgecolor="white", linewidth=2)
    ax.set_title(f"{title}\nH = {H:.3f} bits", fontsize=11, fontweight="bold")
    ax.set_xlabel("Outcome")
    ax.set_ylabel("Probability")
    ax.set_ylim(0, 1)
    ax.set_xticks(range(len(probs)))
    ax.set_xticklabels(labels)
    ax.axhline(1/len(probs), color="gray", linestyle="--", alpha=0.5, label="Uniform")

plt.suptitle("Entropy Reflects How 'Spread Out' a Distribution Is", fontsize=14, fontweight="bold", y=1.02)
plt.tight_layout()
plt.show()

print("\n📊 Key observation: The more uniform (spread out) the distribution,")
print("   the HIGHER the entropy. Maximum entropy = uniform distribution.")

## 5. Maximum Entropy

For $n$ possible outcomes, the **maximum entropy** is achieved when all outcomes are equally likely (uniform distribution):

$$H_{max} = \log(n)$$

This makes intuitive sense: we're most uncertain when we have no reason to expect any outcome over another.

### Normalized Entropy

To compare entropies across systems with different numbers of states, we can **normalize**:

$$H_{normalized} = \frac{H}{H_{max}} = \frac{H}{\log(n)}$$

- Range: 0 (deterministic) to 1 (maximum uncertainty)
- Useful for comparing "how random" different systems are

In [ ]:
def compute_max_entropy(n_states: int, base: float = 2.0) -> float:
    """
    Compute maximum possible entropy for n states.
    
    Parameters
    ----------
    n_states : int
        Number of possible states/outcomes.
    base : float, optional
        Logarithm base. Default is 2 (bits).
    
    Returns
    -------
    float
        Maximum entropy = log(n_states).
    """
    if n_states <= 0:
        raise ValueError("n_states must be positive")
    
    if base == np.e:
        return float(np.log(n_states))
    else:
        return float(np.log(n_states) / np.log(base))


def compute_normalized_entropy(
    probabilities: NDArray[np.float64],
    base: float = 2.0
) -> float:
    """
    Compute entropy normalized by maximum (range 0-1).
    
    Parameters
    ----------
    probabilities : NDArray[np.float64]
        Probability distribution.
    base : float, optional
        Logarithm base. Default is 2.
    
    Returns
    -------
    float
        Normalized entropy in range [0, 1].
    """
    n_states = len(probabilities)
    if n_states <= 1:
        return 0.0
    
    H = compute_entropy_discrete(probabilities, base)
    H_max = compute_max_entropy(n_states, base)
    
    return H / H_max if H_max > 0 else 0.0

In [ ]:
# Visualization 4: Entropy vs distribution "peakedness"

# Create distributions from uniform to very peaked using Dirichlet
n_states = 5
concentration_params = [0.1, 0.5, 1.0, 2.0, 5.0, 10.0, 50.0]

# For reproducibility, we'll create peaked distributions analytically
def create_peaked_distribution(n: int, peak_strength: float) -> NDArray:
    """Create distribution with controllable peakedness."""
    if peak_strength == 0:
        return np.ones(n) / n
    weights = np.exp(-peak_strength * np.abs(np.arange(n) - n//2))
    return weights / np.sum(weights)

peak_values = np.linspace(0, 5, 50)
entropies = []
normalized_entropies = []

for peak in peak_values:
    probs = create_peaked_distribution(n_states, peak)
    entropies.append(compute_entropy_discrete(probs))
    normalized_entropies.append(compute_normalized_entropy(probs))

H_max = compute_max_entropy(n_states)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: Entropy vs peakedness
axes[0].plot(peak_values, entropies, color=COLORS["signal_1"], linewidth=2.5, label="Entropy H")
axes[0].axhline(H_max, color=COLORS["grid"], linestyle="--", linewidth=2, label=f"H_max = {H_max:.3f} bits")
axes[0].axhline(0, color="gray", linestyle=":", linewidth=1)
axes[0].set_xlabel("Peakedness (concentration)", fontsize=12)
axes[0].set_ylabel("Entropy (bits)", fontsize=12)
axes[0].set_title("Entropy Decreases as Distribution Becomes Peaked", fontsize=12, fontweight="bold")
axes[0].legend()
axes[0].grid(True, alpha=0.3)
axes[0].set_ylim(-0.1, H_max + 0.3)

# Right: Example distributions
peak_examples = [0, 1, 3, 5]
colors_ex = [COLORS["signal_1"], COLORS["signal_2"], COLORS["signal_3"], COLORS["signal_4"]]
for peak, color in zip(peak_examples, colors_ex):
    probs = create_peaked_distribution(n_states, peak)
    H = compute_entropy_discrete(probs)
    offset = peak_examples.index(peak) * 0.15
    axes[1].bar(np.arange(n_states) + offset, probs, width=0.15, color=color, 
                label=f"peak={peak}, H={H:.2f}", alpha=0.8)

axes[1].set_xlabel("Outcome", fontsize=12)
axes[1].set_ylabel("Probability", fontsize=12)
axes[1].set_title("Distribution Shapes at Different Peakedness", fontsize=12, fontweight="bold")
axes[1].legend(fontsize=9)
axes[1].set_xticks(np.arange(n_states) + 0.225)
axes[1].set_xticklabels(["A", "B", "C", "D", "E"])

plt.tight_layout()
plt.show()

## 6. Binary Entropy Function

A special and commonly used case is the **binary entropy function**: the entropy of a Bernoulli variable (two outcomes) as a function of a single probability $p$:

$$H(p) = -p \log_2(p) - (1-p) \log_2(1-p)$$

### Properties

- **H(0) = H(1) = 0**: Deterministic outcomes → no uncertainty
- **H(0.5) = 1 bit**: Maximum uncertainty for binary variable
- **Symmetric**: H(p) = H(1-p)

This function is useful for understanding information in yes/no questions and binary classifications.

In [ ]:
def binary_entropy(p: Union[float, NDArray[np.float64]]) -> Union[float, NDArray[np.float64]]:
    """
    Compute binary entropy function H(p) in bits.
    
    Parameters
    ----------
    p : float or NDArray
        Probability value(s) in range [0, 1].
    
    Returns
    -------
    float or NDArray
        Binary entropy H(p) = -p*log2(p) - (1-p)*log2(1-p).
    """
    p = np.asarray(p)
    
    # Handle edge cases
    result = np.zeros_like(p, dtype=float)
    
    # Only compute for valid probabilities (not 0 or 1)
    valid = (p > 0) & (p < 1)
    p_valid = p[valid]
    result[valid] = -p_valid * np.log2(p_valid) - (1 - p_valid) * np.log2(1 - p_valid)
    
    # Return scalar if input was scalar
    if result.ndim == 0:
        return float(result)
    return result

In [ ]:
# Visualization 5: Binary entropy function

p_values = np.linspace(0, 1, 500)
H_binary = binary_entropy(p_values)

fig, ax = plt.subplots(figsize=(10, 6))

ax.plot(p_values, H_binary, color=COLORS["signal_1"], linewidth=3)
ax.fill_between(p_values, H_binary, alpha=0.2, color=COLORS["signal_1"])

# Mark key points
ax.scatter([0.5], [1.0], color=COLORS["signal_2"], s=150, zorder=5, edgecolor="white", linewidth=2)
ax.annotate("Maximum: H(0.5) = 1 bit", xy=(0.5, 1.0), xytext=(0.65, 0.9),
            fontsize=11, fontweight="bold",
            arrowprops=dict(arrowstyle="->", color="black"))

# Mark zero points
ax.scatter([0, 1], [0, 0], color=COLORS["signal_3"], s=100, zorder=5)
ax.annotate("H(0) = 0\n(certain outcome)", xy=(0, 0), xytext=(0.08, 0.15), fontsize=10)
ax.annotate("H(1) = 0\n(certain outcome)", xy=(1, 0), xytext=(0.75, 0.15), fontsize=10)

ax.set_xlabel("Probability p (of one outcome)", fontsize=12)
ax.set_ylabel("Binary Entropy H(p) [bits]", fontsize=12)
ax.set_title("Binary Entropy Function: Uncertainty in Yes/No Questions", fontsize=14, fontweight="bold")
ax.set_xlim(-0.02, 1.02)
ax.set_ylim(-0.05, 1.1)
ax.grid(True, alpha=0.3)

# Add symmetry annotation
ax.annotate("", xy=(0.3, 0.88), xytext=(0.7, 0.88),
            arrowprops=dict(arrowstyle="<->", color="gray", lw=1.5))
ax.text(0.5, 0.92, "Symmetric: H(p) = H(1-p)", ha="center", fontsize=10, color="gray")

plt.tight_layout()
plt.show()

print("\n💡 A fair coin flip gives exactly 1 bit of information.")
print("   This is why 'bit' is the fundamental unit of information!")

## 7. Entropy of Continuous Variables

Neural signals like EEG are **continuous** — they can take any value in a range. How do we compute entropy for continuous variables?

### The Problem

Continuous variables have infinitely many possible values. We can't directly sum over infinite outcomes!

### Solution 1: Discretization (Binning)

The practical approach:
1. Divide the value range into **bins**
2. Count samples in each bin
3. Convert counts to probabilities
4. Compute discrete entropy

This is the most common approach for neural signal analysis.

### Solution 2: Differential Entropy

The theoretical continuous analogue:

$$h(X) = -\int p(x) \log p(x) \, dx$$

⚠️ **Important differences** from discrete entropy:
- Can be **negative** (unlike discrete entropy)
- Depends on the **units** of measurement
- Not directly comparable across different variable types

For most practical neural signal analysis, we use the **binning approach**.

In [ ]:
# Visualization 6: Continuous distribution and discretization

np.random.seed(42)
continuous_data = np.random.normal(0, 1, 5000)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: Continuous distribution (KDE)
x_range = np.linspace(-4, 4, 200)
kde = stats.gaussian_kde(continuous_data)
axes[0].fill_between(x_range, kde(x_range), alpha=0.5, color=COLORS["signal_1"])
axes[0].plot(x_range, kde(x_range), color=COLORS["signal_1"], linewidth=2)
axes[0].set_xlabel("Value", fontsize=12)
axes[0].set_ylabel("Probability Density", fontsize=12)
axes[0].set_title("Continuous Distribution\n(Infinitely many possible values)", fontsize=12, fontweight="bold")
axes[0].set_xlim(-4, 4)

# Right: Discretized (binned) version
n_bins = 20
counts, bin_edges, _ = axes[1].hist(continuous_data, bins=n_bins, color=COLORS["signal_2"], 
                                     edgecolor="white", linewidth=1, alpha=0.8, density=True)
axes[1].set_xlabel("Value", fontsize=12)
axes[1].set_ylabel("Probability Density", fontsize=12)
axes[1].set_title(f"Discretized into {n_bins} Bins\n(Finite number of states)", fontsize=12, fontweight="bold")
axes[1].set_xlim(-4, 4)

# Add annotation
axes[1].annotate("Each bin becomes\na discrete state", xy=(1.5, 0.1), xytext=(2.5, 0.2),
                 fontsize=10, arrowprops=dict(arrowstyle="->", color="black"))

plt.suptitle("Binning Converts Continuous to Discrete", fontsize=14, fontweight="bold", y=1.02)
plt.tight_layout()
plt.show()

## 8. Binning Strategies

The choice of binning strategy critically affects entropy estimation. Let's explore the options.

### The Binning Process

1. **Choose number of bins** (critical choice!)
2. **Define bin edges** (uniform width or adaptive)
3. **Count samples** in each bin
4. **Convert counts to probabilities**
5. **Compute discrete entropy**

### Binning Methods

1. **Uniform width**: Equal-sized bins across data range (most common)
2. **Uniform count (equiprobable)**: Bins with equal number of samples
3. **Adaptive**: Data-driven bin edges (e.g., based on data structure)

### Choosing Number of Bins

- **Too few bins**: Lose information, underestimate entropy
- **Too many bins**: Sparse bins, biased estimation, overestimate entropy

Common rules of thumb:
- **√n rule**: bins = √(number of samples)
- **Sturges' rule**: bins = 1 + log₂(n)
- **Freedman-Diaconis**: bins based on interquartile range

In [ ]:
def optimal_n_bins(n_samples: int, method: str = "sturges") -> int:
    """
    Compute optimal number of bins for entropy estimation.
    
    Parameters
    ----------
    n_samples : int
        Number of data samples.
    method : str, optional
        Method for determining bins: "sturges", "sqrt", "rice".
        Default is "sturges".
    
    Returns
    -------
    int
        Recommended number of bins.
    """
    if n_samples <= 0:
        raise ValueError("n_samples must be positive")
    
    if method == "sturges":
        return max(1, int(np.ceil(1 + np.log2(n_samples))))
    elif method == "sqrt":
        return max(1, int(np.ceil(np.sqrt(n_samples))))
    elif method == "rice":
        return max(1, int(np.ceil(2 * n_samples ** (1/3))))
    else:
        raise ValueError(f"Unknown method: {method}. Use 'sturges', 'sqrt', or 'rice'.")


def compute_entropy_continuous(
    signal: NDArray[np.float64],
    n_bins: Union[int, str] = "auto",
    method: str = "uniform"
) -> Tuple[float, int]:
    """
    Estimate entropy of a continuous signal via binning.
    
    Parameters
    ----------
    signal : NDArray[np.float64]
        Continuous signal to analyze.
    n_bins : int or str, optional
        Number of bins, or "auto"/"sturges"/"sqrt" for automatic.
        Default is "auto" (uses Sturges' rule).
    method : str, optional
        Binning method: "uniform" (equal width) or "equiprobable" (equal count).
        Default is "uniform".
    
    Returns
    -------
    Tuple[float, int]
        (entropy in bits, actual number of bins used)
    """
    n_samples = len(signal)
    
    # Determine number of bins
    if isinstance(n_bins, str):
        if n_bins == "auto" or n_bins == "sturges":
            actual_bins = optimal_n_bins(n_samples, "sturges")
        elif n_bins == "sqrt":
            actual_bins = optimal_n_bins(n_samples, "sqrt")
        else:
            actual_bins = optimal_n_bins(n_samples, "sturges")
    else:
        actual_bins = int(n_bins)
    
    # Compute histogram
    if method == "uniform":
        counts, _ = np.histogram(signal, bins=actual_bins)
    elif method == "equiprobable":
        # Create bins with equal number of samples
        percentiles = np.linspace(0, 100, actual_bins + 1)
        bin_edges = np.percentile(signal, percentiles)
        counts, _ = np.histogram(signal, bins=bin_edges)
    else:
        raise ValueError(f"Unknown method: {method}")
    
    # Compute entropy from counts
    entropy = compute_entropy_from_counts(counts)
    
    return entropy, actual_bins

In [ ]:
# Visualization 7: Effect of binning on entropy estimation

np.random.seed(42)
signal = np.random.normal(0, 1, 1000)

bin_counts = [5, 20, 50, 200]
fig, axes = plt.subplots(2, 2, figsize=(12, 10))
axes = axes.flatten()

for ax, n_bins in zip(axes, bin_counts):
    entropy, _ = compute_entropy_continuous(signal, n_bins=n_bins)
    H_max = compute_max_entropy(n_bins)
    
    ax.hist(signal, bins=n_bins, color=COLORS["signal_1"], edgecolor="white", alpha=0.8)
    ax.set_title(f"{n_bins} bins: H = {entropy:.3f} bits (H_max = {H_max:.2f})", 
                 fontsize=12, fontweight="bold")
    ax.set_xlabel("Value")
    ax.set_ylabel("Count")
    ax.set_xlim(-4, 4)

plt.suptitle("How Binning Choice Affects Entropy Estimate", fontsize=14, fontweight="bold", y=1.02)
plt.tight_layout()
plt.show()

# Show entropy vs bins
print("\n📊 Entropy vs Number of Bins:")
print("-" * 40)
for n_bins in [5, 10, 20, 50, 100, 200]:
    H, _ = compute_entropy_continuous(signal, n_bins=n_bins)
    H_max = compute_max_entropy(n_bins)
    print(f"  {n_bins:3d} bins: H = {H:.3f} bits  (H_max = {H_max:.2f}, ratio = {H/H_max:.3f})")

print("\n⚠️ Note: Entropy increases with more bins, but normalized ratio stays similar.")

## 9. Bias in Entropy Estimation

Entropy estimated from finite samples is **biased** — it tends to **underestimate** the true entropy.

### Why the Bias?

- With finite samples, we can't observe all possible outcomes
- Empty bins are treated as zero probability
- Underestimation is more severe with more bins or fewer samples

### Miller-Madow Correction

A simple bias correction:

$$\hat{H}_{corrected} = \hat{H} + \frac{m - 1}{2n \ln(b)}$$

Where:
- $m$ = number of bins with non-zero counts
- $n$ = total number of samples
- $b$ = logarithm base

This is a first-order correction that helps reduce bias.

In [ ]:
def compute_entropy_miller_madow(
    signal: NDArray[np.float64],
    n_bins: int,
    base: float = 2.0
) -> float:
    """
    Compute entropy with Miller-Madow bias correction.
    
    Parameters
    ----------
    signal : NDArray[np.float64]
        Continuous signal to analyze.
    n_bins : int
        Number of bins for discretization.
    base : float, optional
        Logarithm base. Default is 2 (bits).
    
    Returns
    -------
    float
        Bias-corrected entropy estimate.
    """
    n_samples = len(signal)
    
    # Compute histogram
    counts, _ = np.histogram(signal, bins=n_bins)
    
    # Number of non-empty bins
    m = np.sum(counts > 0)
    
    # Raw entropy
    H_raw = compute_entropy_from_counts(counts, base)
    
    # Miller-Madow correction
    if base == np.e:
        correction = (m - 1) / (2 * n_samples)
    else:
        correction = (m - 1) / (2 * n_samples * np.log(base))
    
    return H_raw + correction

In [ ]:
# Visualization 8: Bias correction demonstration

np.random.seed(42)

# For uniform distribution on [0, 1] with n_bins uniform bins,
# true entropy = log2(n_bins)
n_bins_sim = 20
true_entropy = np.log2(n_bins_sim)

sample_sizes = [50, 100, 200, 500, 1000, 2000, 5000]
n_trials = 100

raw_means, raw_stds = [], []
corrected_means, corrected_stds = [], []

for n in sample_sizes:
    raw_estimates = []
    corrected_estimates = []
    
    for _ in range(n_trials):
        data = np.random.uniform(0, 1, n)
        H_raw, _ = compute_entropy_continuous(data, n_bins=n_bins_sim)
        H_corrected = compute_entropy_miller_madow(data, n_bins_sim)
        raw_estimates.append(H_raw)
        corrected_estimates.append(H_corrected)
    
    raw_means.append(np.mean(raw_estimates))
    raw_stds.append(np.std(raw_estimates))
    corrected_means.append(np.mean(corrected_estimates))
    corrected_stds.append(np.std(corrected_estimates))

fig, ax = plt.subplots(figsize=(12, 6))

# Plot raw estimates
ax.errorbar(sample_sizes, raw_means, yerr=raw_stds, 
            color=COLORS["signal_1"], linewidth=2, marker="o", markersize=8,
            capsize=5, label="Raw estimate (biased)")

# Plot corrected estimates
ax.errorbar(sample_sizes, corrected_means, yerr=corrected_stds,
            color=COLORS["signal_2"], linewidth=2, marker="s", markersize=8,
            capsize=5, label="Miller-Madow corrected")

# True value
ax.axhline(true_entropy, color=COLORS["grid"], linestyle="--", linewidth=2, 
           label=f"True entropy = {true_entropy:.3f} bits")

ax.set_xscale("log")
ax.set_xlabel("Number of Samples", fontsize=12)
ax.set_ylabel("Estimated Entropy (bits)", fontsize=12)
ax.set_title("Entropy Estimation Bias and Correction", fontsize=14, fontweight="bold")
ax.legend(loc="lower right", fontsize=11)
ax.grid(True, alpha=0.3)
ax.set_ylim(true_entropy - 1, true_entropy + 0.3)

plt.tight_layout()
plt.show()

print(f"\n📊 True entropy for uniform distribution with {n_bins_sim} bins: {true_entropy:.4f} bits")
print(f"   With 100 samples: raw = {raw_means[1]:.4f}, corrected = {corrected_means[1]:.4f}")
print(f"   With 5000 samples: raw = {raw_means[-1]:.4f}, corrected = {corrected_means[-1]:.4f}")
print("\n✓ The correction helps, especially with fewer samples!")

---

Excellent work so far! Let's continue to the second part of the notebook.

---

In [ ]:
# Visualization 9: Entropy of different signal types

np.random.seed(42)
fs = 256  # Sampling rate (Hz)
duration = 4  # seconds
t = np.arange(0, duration, 1/fs)
n_samples = len(t)

# Generate different signal types
signals = {}

# 1. Pure sine wave (very predictable)
signals["Pure Sine (10 Hz)"] = np.sin(2 * np.pi * 10 * t)

# 2. Sine with noise (somewhat predictable)
signals["Sine + Noise"] = np.sin(2 * np.pi * 10 * t) + 0.3 * np.random.randn(n_samples)

# 3. Mixed frequencies (more complex)
signals["Mixed Frequencies"] = (np.sin(2 * np.pi * 8 * t) + 
                                 0.7 * np.sin(2 * np.pi * 12 * t) + 
                                 0.5 * np.sin(2 * np.pi * 20 * t))

# 4. White noise (maximally unpredictable)
signals["White Noise"] = np.random.randn(n_samples)

# 5. Simulated alpha rhythm (realistic EEG-like)
alpha = np.sin(2 * np.pi * 10 * t) * (1 + 0.3 * np.sin(2 * np.pi * 0.5 * t))
signals["Alpha-like"] = alpha + 0.2 * np.random.randn(n_samples)

# Compute entropy for each
n_bins_neural = optimal_n_bins(n_samples, "sturges")
colors_signals = [COLORS["signal_1"], COLORS["signal_2"], COLORS["signal_3"], 
                  COLORS["signal_4"], COLORS["signal_5"]]

fig, axes = plt.subplots(len(signals), 2, figsize=(14, 12))

for idx, (name, signal) in enumerate(signals.items()):
    # Normalize signal
    signal_norm = (signal - np.mean(signal)) / np.std(signal)
    
    # Compute entropy
    H, actual_bins = compute_entropy_continuous(signal_norm, n_bins=n_bins_neural)
    H_max = compute_max_entropy(actual_bins)
    H_norm = H / H_max
    
    # Left: Time series
    axes[idx, 0].plot(t[:256], signal_norm[:256], color=colors_signals[idx], linewidth=1)
    axes[idx, 0].set_ylabel(name, fontsize=10, fontweight="bold")
    axes[idx, 0].set_xlim(0, 1)
    if idx == len(signals) - 1:
        axes[idx, 0].set_xlabel("Time (s)", fontsize=11)
    if idx == 0:
        axes[idx, 0].set_title("Signal (1 second)", fontsize=12, fontweight="bold")
    
    # Right: Distribution
    axes[idx, 1].hist(signal_norm, bins=n_bins_neural, color=colors_signals[idx], 
                      edgecolor="white", alpha=0.8, density=True)
    axes[idx, 1].set_title(f"H = {H:.2f} bits (normalized: {H_norm:.2f})", fontsize=11)
    if idx == len(signals) - 1:
        axes[idx, 1].set_xlabel("Amplitude", fontsize=11)
    if idx == 0:
        axes[idx, 1].set_title(f"Distribution — H = {H:.2f} bits (norm: {H_norm:.2f})", 
                               fontsize=12, fontweight="bold")

plt.suptitle("Entropy of Different Signal Types", fontsize=14, fontweight="bold", y=1.01)
plt.tight_layout()
plt.show()

print("\n📊 Key observations:")
print("   • Pure sine has LOWER entropy (predictable, concentrated distribution)")
print("   • White noise has HIGHER entropy (unpredictable, uniform-like distribution)")
print("   • Real neural signals fall in between!")

## 11. Spectral Entropy

**Spectral entropy** applies the entropy concept to the **frequency domain**. Instead of measuring uncertainty in amplitude values, it measures uncertainty in the distribution of power across frequencies.

### Definition

Given a power spectral density (PSD) normalized to sum to 1 (making it a probability distribution):

$$H_{spectral} = -\sum_{f} P(f) \log_2 P(f)$$

Where $P(f)$ is the normalized power at frequency $f$.

### Interpretation

- **High spectral entropy**: Power spread across many frequencies → "broadband" signal
- **Low spectral entropy**: Power concentrated at few frequencies → "narrowband" signal (e.g., strong oscillation)

### Normalized Spectral Entropy

$$H_{spectral,norm} = \frac{H_{spectral}}{\log_2(N_{freq})}$$

Range: 0 (single frequency) to 1 (flat spectrum/white noise)

In [ ]:
def compute_spectral_entropy(
    signal: NDArray[np.float64],
    fs: float,
    nperseg: int = 256,
    freq_range: Optional[Tuple[float, float]] = None,
    normalize: bool = True
) -> Tuple[float, NDArray[np.float64], NDArray[np.float64]]:
    """
    Compute spectral entropy of a signal.
    
    Parameters
    ----------
    signal : NDArray[np.float64]
        Time series signal.
    fs : float
        Sampling frequency in Hz.
    nperseg : int, optional
        Length of each segment for Welch's method. Default is 256.
    freq_range : Tuple[float, float], optional
        Frequency range (fmin, fmax) to consider. Default is None (all frequencies).
    normalize : bool, optional
        Whether to normalize by maximum entropy. Default is True.
    
    Returns
    -------
    Tuple[float, NDArray, NDArray]
        (spectral_entropy, frequencies, psd)
    """
    # Compute PSD using Welch's method
    freqs, psd = welch(signal, fs=fs, nperseg=min(nperseg, len(signal)))
    
    # Apply frequency range if specified
    if freq_range is not None:
        mask = (freqs >= freq_range[0]) & (freqs <= freq_range[1])
        freqs = freqs[mask]
        psd = psd[mask]
    
    # Normalize PSD to make it a probability distribution
    psd_norm = psd / np.sum(psd)
    
    # Remove zeros
    psd_valid = psd_norm[psd_norm > 0]
    
    # Compute entropy
    H_spectral = -np.sum(psd_valid * np.log2(psd_valid))
    
    # Normalize if requested
    if normalize:
        H_max = np.log2(len(psd_valid))
        if H_max > 0:
            H_spectral = H_spectral / H_max
    
    return float(H_spectral), freqs, psd

In [ ]:
# Visualization 10: Spectral entropy comparison

np.random.seed(42)
fs = 256
duration = 10
t = np.arange(0, duration, 1/fs)
n_samples = len(t)

# Create signals with different spectral characteristics
spectral_signals = {}

# 1. Pure 10 Hz sine (narrowband)
spectral_signals["Pure 10 Hz"] = np.sin(2 * np.pi * 10 * t)

# 2. Alpha band oscillation (8-12 Hz)
alpha_signal = (np.sin(2 * np.pi * 8 * t) + 
                np.sin(2 * np.pi * 10 * t) + 
                np.sin(2 * np.pi * 12 * t)) / 3
spectral_signals["Alpha Band\n(8-12 Hz)"] = alpha_signal

# 3. Multiple bands
multi_band = (np.sin(2 * np.pi * 10 * t) +  # alpha
              0.7 * np.sin(2 * np.pi * 20 * t) +  # beta
              0.5 * np.sin(2 * np.pi * 6 * t))  # theta
spectral_signals["Multi-band"] = multi_band

# 4. White noise (flat spectrum)
spectral_signals["White Noise"] = np.random.randn(n_samples)

# 5. Pink noise (1/f spectrum)
# Generate 1/f noise using FFT
fft_freqs = np.fft.rfftfreq(n_samples, 1/fs)
fft_freqs[0] = 1  # Avoid division by zero
pink_spectrum = 1 / np.sqrt(fft_freqs)
pink_phase = np.random.uniform(0, 2*np.pi, len(fft_freqs))
pink_fft = pink_spectrum * np.exp(1j * pink_phase)
pink_noise = np.fft.irfft(pink_fft, n_samples)
spectral_signals["Pink Noise (1/f)"] = pink_noise

# Compute and plot
fig, axes = plt.subplots(len(spectral_signals), 2, figsize=(14, 14))
colors_spec = [COLORS["signal_1"], COLORS["signal_2"], COLORS["signal_3"], 
               COLORS["signal_4"], COLORS["signal_5"]]

for idx, (name, signal) in enumerate(spectral_signals.items()):
    H_spec, freqs, psd = compute_spectral_entropy(signal, fs, freq_range=(1, 50))
    
    # Left: PSD
    axes[idx, 0].semilogy(freqs, psd, color=colors_spec[idx], linewidth=2)
    axes[idx, 0].set_ylabel(name, fontsize=10, fontweight="bold")
    axes[idx, 0].set_xlim(0, 50)
    axes[idx, 0].grid(True, alpha=0.3)
    if idx == len(spectral_signals) - 1:
        axes[idx, 0].set_xlabel("Frequency (Hz)", fontsize=11)
    if idx == 0:
        axes[idx, 0].set_title("Power Spectral Density", fontsize=12, fontweight="bold")
    
    # Right: Normalized PSD (probability distribution)
    psd_norm = psd / np.sum(psd)
    axes[idx, 1].fill_between(freqs, psd_norm, alpha=0.5, color=colors_spec[idx])
    axes[idx, 1].plot(freqs, psd_norm, color=colors_spec[idx], linewidth=2)
    axes[idx, 1].set_xlim(0, 50)
    axes[idx, 1].set_title(f"H_spectral = {H_spec:.3f}", fontsize=11)
    if idx == len(spectral_signals) - 1:
        axes[idx, 1].set_xlabel("Frequency (Hz)", fontsize=11)
    if idx == 0:
        axes[idx, 1].set_title(f"Normalized PSD — H = {H_spec:.3f}", fontsize=12, fontweight="bold")

plt.suptitle("Spectral Entropy: How Spread is Power Across Frequencies?", 
             fontsize=14, fontweight="bold", y=1.01)
plt.tight_layout()
plt.show()

print("\n📊 Spectral Entropy Interpretation:")
print("   • Pure tone (1 freq): LOW spectral entropy")
print("   • White noise (all freqs equally): HIGH spectral entropy")
print("   • Neural signals: Between extremes, depends on oscillatory content")

## 12. Sample Entropy

**Sample entropy** is a measure of signal **complexity** that doesn't require binning. It measures how "self-similar" a signal is by comparing patterns at different time points.

### Concept

Sample entropy asks: *If two segments of the signal are similar now, how likely are they to remain similar if we extend them by one sample?*

- **Low sample entropy**: Signal has repeating patterns → predictable
- **High sample entropy**: Signal patterns don't repeat → unpredictable

### Parameters

- **m** (embedding dimension): Length of patterns to compare (typically 2)
- **r** (tolerance): How close patterns must be to be "similar" (typically 0.2 × std)

### Advantages Over Binning-Based Entropy

1. No binning required → no bias from bin choice
2. Captures temporal structure (pattern matching)
3. Robust to noise and signal length
4. Commonly used in physiological signal analysis

In [ ]:
def compute_sample_entropy(
    signal: NDArray[np.float64],
    m: int = 2,
    r: Optional[float] = None
) -> float:
    """
    Compute sample entropy of a time series.
    
    Parameters
    ----------
    signal : NDArray[np.float64]
        Time series signal.
    m : int, optional
        Embedding dimension. Default is 2.
    r : float, optional
        Tolerance for pattern matching. Default is 0.2 * std(signal).
    
    Returns
    -------
    float
        Sample entropy value.
    
    Notes
    -----
    Higher values indicate more complexity/irregularity.
    Lower values indicate more self-similarity/regularity.
    """
    N = len(signal)
    
    if r is None:
        r = 0.2 * np.std(signal)
    
    def count_matches(template_length: int) -> int:
        """Count pairs of matching templates."""
        templates = np.array([
            signal[i:i + template_length] 
            for i in range(N - template_length)
        ])
        
        count = 0
        n_templates = len(templates)
        
        for i in range(n_templates):
            for j in range(i + 1, n_templates):
                # Chebyshev distance (max absolute difference)
                if np.max(np.abs(templates[i] - templates[j])) <= r:
                    count += 1
        
        return count
    
    # Count matches for m and m+1
    A = count_matches(m + 1)  # matches for length m+1
    B = count_matches(m)      # matches for length m
    
    # Sample entropy
    if A == 0 or B == 0:
        return np.inf  # No matches found
    
    return -np.log(A / B)

In [ ]:
# Visualization 11: Sample entropy for different signals

np.random.seed(42)

# Create short signals for sample entropy (computation is O(n²))
n_samples_se = 500
t_se = np.arange(n_samples_se) / 256

sample_entropy_signals = {}

# 1. Regular sine wave
sample_entropy_signals["Regular Sine"] = np.sin(2 * np.pi * 10 * t_se)

# 2. Sine with amplitude modulation
sample_entropy_signals["AM Sine"] = np.sin(2 * np.pi * 10 * t_se) * (1 + 0.5 * np.sin(2 * np.pi * 0.5 * t_se))

# 3. Chaotic signal (logistic map)
x_logistic = np.zeros(n_samples_se)
x_logistic[0] = 0.1
r_param = 3.9  # Chaotic regime
for i in range(1, n_samples_se):
    x_logistic[i] = r_param * x_logistic[i-1] * (1 - x_logistic[i-1])
sample_entropy_signals["Chaotic\n(Logistic Map)"] = x_logistic

# 4. Random walk
random_walk = np.cumsum(np.random.randn(n_samples_se))
sample_entropy_signals["Random Walk"] = random_walk

# 5. White noise
sample_entropy_signals["White Noise"] = np.random.randn(n_samples_se)

# Compute sample entropy
fig, axes = plt.subplots(len(sample_entropy_signals), 1, figsize=(14, 12))
colors_se = [COLORS["signal_1"], COLORS["signal_2"], COLORS["signal_3"], 
             COLORS["signal_4"], COLORS["signal_5"]]

sample_entropies = []

for idx, (name, signal) in enumerate(sample_entropy_signals.items()):
    # Normalize
    signal_norm = (signal - np.mean(signal)) / np.std(signal)
    
    # Compute sample entropy
    se = compute_sample_entropy(signal_norm, m=2)
    sample_entropies.append((name.replace("\n", " "), se))
    
    # Plot
    axes[idx].plot(t_se, signal_norm, color=colors_se[idx], linewidth=1)
    axes[idx].set_ylabel(name, fontsize=10, fontweight="bold")
    axes[idx].set_xlim(0, t_se[-1])
    
    # Add sample entropy annotation
    axes[idx].text(0.98, 0.95, f"SampEn = {se:.3f}", transform=axes[idx].transAxes,
                   fontsize=11, fontweight="bold", ha="right", va="top",
                   bbox=dict(boxstyle="round", facecolor="white", alpha=0.8))
    
    if idx == len(sample_entropy_signals) - 1:
        axes[idx].set_xlabel("Time (s)", fontsize=11)

plt.suptitle("Sample Entropy: Measuring Pattern Regularity", fontsize=14, fontweight="bold", y=1.01)
plt.tight_layout()
plt.show()

# Summary bar chart
print("\n📊 Sample Entropy Summary:")
print("-" * 50)
for name, se in sample_entropies:
    bar = "█" * int(se * 10) if se < 3 else "█" * 30 + "..."
    print(f"  {name:<24}: {se:.3f}  {bar}")

print("\n✓ Regular signals have LOW sample entropy (patterns repeat)")
print("✓ Complex/random signals have HIGH sample entropy (no repeating patterns)")

## 13. Entropy in Hyperscanning Context

In hyperscanning, we record brain signals from **multiple people** simultaneously. How does entropy help us understand inter-brain connectivity?

### Individual Signal Entropy

- Characterizes the **complexity** of each person's brain activity
- Can reveal different cognitive states or engagement levels
- Baseline measure before computing connectivity

### Entropy and Connectivity (Preview)

The real power of entropy for connectivity comes in the next notebooks:

1. **Mutual Information (D02)**: How much entropy is *shared* between two signals?
   - High MI = knowing one signal reduces uncertainty about the other
   
2. **Transfer Entropy (D03)**: How much does the past of one signal reduce entropy of another?
   - Reveals *directional* information flow between brains

### Entropy as a Preprocessing Tool

- Identify noisy channels (unexpectedly high entropy)
- Detect artifacts (abrupt entropy changes)
- Characterize signal quality before connectivity analysis

In [ ]:
# Visualization 12: Entropy in a simulated hyperscanning scenario

np.random.seed(42)
fs = 256
duration = 10
t = np.arange(0, duration, 1/fs)
n_samples = len(t)

# Simulate two subjects with different engagement levels
# Subject 1: Engaged (more structured alpha oscillation)
alpha_1 = 2 * np.sin(2 * np.pi * 10 * t) * (1 + 0.3 * np.sin(2 * np.pi * 0.2 * t))
noise_1 = 0.3 * np.random.randn(n_samples)
subject_1 = alpha_1 + noise_1

# Subject 2: Less engaged (weaker oscillation, more noise)
alpha_2 = 0.8 * np.sin(2 * np.pi * 10 * t + np.pi/4)
noise_2 = 0.8 * np.random.randn(n_samples)
subject_2 = alpha_2 + noise_2

# Normalize
subject_1 = (subject_1 - np.mean(subject_1)) / np.std(subject_1)
subject_2 = (subject_2 - np.mean(subject_2)) / np.std(subject_2)

# Compute entropies
n_bins = optimal_n_bins(n_samples)
H1, _ = compute_entropy_continuous(subject_1, n_bins=n_bins)
H2, _ = compute_entropy_continuous(subject_2, n_bins=n_bins)
H_max = compute_max_entropy(n_bins)

H1_spec, _, psd1 = compute_spectral_entropy(subject_1, fs)
H2_spec, freqs, psd2 = compute_spectral_entropy(subject_2, fs)

fig, axes = plt.subplots(2, 3, figsize=(15, 8))

# Row 1: Subject 1
axes[0, 0].plot(t[:512], subject_1[:512], color=COLORS["signal_1"], linewidth=1)
axes[0, 0].set_title("Subject 1: Engaged\n(Strong Alpha)", fontsize=11, fontweight="bold")
axes[0, 0].set_ylabel("Amplitude", fontsize=10)
axes[0, 0].set_xlabel("Time (s)", fontsize=10)
axes[0, 0].set_xlim(0, 2)

axes[0, 1].hist(subject_1, bins=n_bins, color=COLORS["signal_1"], edgecolor="white", alpha=0.8, density=True)
axes[0, 1].set_title(f"Distribution\nH = {H1:.2f} bits ({H1/H_max:.2f} norm)", fontsize=11, fontweight="bold")
axes[0, 1].set_xlabel("Amplitude", fontsize=10)

psd1_norm = psd1 / np.sum(psd1)
axes[0, 2].fill_between(freqs, psd1_norm, alpha=0.5, color=COLORS["signal_1"])
axes[0, 2].plot(freqs, psd1_norm, color=COLORS["signal_1"], linewidth=2)
axes[0, 2].set_title(f"Spectrum\nH_spec = {H1_spec:.3f}", fontsize=11, fontweight="bold")
axes[0, 2].set_xlabel("Frequency (Hz)", fontsize=10)
axes[0, 2].set_xlim(0, 40)

# Row 2: Subject 2
axes[1, 0].plot(t[:512], subject_2[:512], color=COLORS["signal_2"], linewidth=1)
axes[1, 0].set_title("Subject 2: Less Engaged\n(Weak Alpha, More Noise)", fontsize=11, fontweight="bold")
axes[1, 0].set_ylabel("Amplitude", fontsize=10)
axes[1, 0].set_xlabel("Time (s)", fontsize=10)
axes[1, 0].set_xlim(0, 2)

axes[1, 1].hist(subject_2, bins=n_bins, color=COLORS["signal_2"], edgecolor="white", alpha=0.8, density=True)
axes[1, 1].set_title(f"Distribution\nH = {H2:.2f} bits ({H2/H_max:.2f} norm)", fontsize=11, fontweight="bold")
axes[1, 1].set_xlabel("Amplitude", fontsize=10)

psd2_norm = psd2 / np.sum(psd2)
axes[1, 2].fill_between(freqs, psd2_norm, alpha=0.5, color=COLORS["signal_2"])
axes[1, 2].plot(freqs, psd2_norm, color=COLORS["signal_2"], linewidth=2)
axes[1, 2].set_title(f"Spectrum\nH_spec = {H2_spec:.3f}", fontsize=11, fontweight="bold")
axes[1, 2].set_xlabel("Frequency (Hz)", fontsize=10)
axes[1, 2].set_xlim(0, 40)

plt.suptitle("Hyperscanning: Characterizing Individual Signal Properties with Entropy", 
             fontsize=14, fontweight="bold", y=1.02)
plt.tight_layout()
plt.show()

print("\n📊 Entropy Comparison:")
print(f"   Subject 1 (engaged):     H_amplitude = {H1:.3f}, H_spectral = {H1_spec:.3f}")
print(f"   Subject 2 (less engaged): H_amplitude = {H2:.3f}, H_spectral = {H2_spec:.3f}")
print("\n💡 The less engaged subject shows:")
print("   • Higher amplitude entropy (more random)")
print("   • Higher spectral entropy (power spread across frequencies)")
print("\n→ Next step: Quantify SHARED information between subjects (Mutual Information, D02)")

## 14. Exercises

Test your understanding with these exercises!

### Exercise 1: Entropy Calculation
Given a probability distribution [0.25, 0.25, 0.25, 0.25], calculate:
1. The Shannon entropy in bits
2. Is this maximum entropy for 4 states?
3. What distribution would give minimum entropy?

### Exercise 2: Binning Impact
Generate a uniform random signal of 1000 samples. Compute entropy with 10, 50, and 100 bins.
- How does entropy change with bin count?
- What is the normalized entropy in each case?

### Exercise 3: Spectral Entropy
Create two signals: (a) pure 10 Hz sine, (b) sum of 5, 10, 15, 20 Hz sines.
- Which has higher spectral entropy?
- What does this tell you about the signals?

### Exercise 4: Entropy and Neural States
Think about how entropy might differ between:
- Eyes open vs. eyes closed (alpha rhythm)
- Rest vs. cognitive task
- Awake vs. sleep stages

What entropy patterns would you predict?

In [ ]:
# Exercise solutions (try yourself first!)

# Exercise 1
print("=" * 60)
print("EXERCISE 1: Entropy Calculation")
print("=" * 60)
uniform_4 = np.array([0.25, 0.25, 0.25, 0.25])
H_uniform = compute_entropy_discrete(uniform_4)
H_max_4 = compute_max_entropy(4)
print(f"1. H = {H_uniform:.4f} bits")
print(f"2. H_max for 4 states = log₂(4) = {H_max_4:.4f} bits")
print(f"   → Yes! Uniform distribution achieves maximum entropy.")
print(f"3. Minimum entropy: [1, 0, 0, 0] → H = 0 (deterministic)")

# Exercise 2
print("\n" + "=" * 60)
print("EXERCISE 2: Binning Impact")
print("=" * 60)
np.random.seed(42)
uniform_signal = np.random.uniform(0, 1, 1000)
for n_bins in [10, 50, 100]:
    H, _ = compute_entropy_continuous(uniform_signal, n_bins=n_bins)
    H_max = compute_max_entropy(n_bins)
    print(f"  {n_bins:3d} bins: H = {H:.3f} bits, H_max = {H_max:.3f}, normalized = {H/H_max:.4f}")
print("  → Entropy increases with bins, but normalized entropy stays ~constant (~1.0)")

# Exercise 3
print("\n" + "=" * 60)
print("EXERCISE 3: Spectral Entropy")
print("=" * 60)
t_ex = np.arange(0, 5, 1/256)
pure_sine = np.sin(2 * np.pi * 10 * t_ex)
multi_sine = (np.sin(2 * np.pi * 5 * t_ex) + np.sin(2 * np.pi * 10 * t_ex) + 
              np.sin(2 * np.pi * 15 * t_ex) + np.sin(2 * np.pi * 20 * t_ex))
H_pure, _, _ = compute_spectral_entropy(pure_sine, 256)
H_multi, _, _ = compute_spectral_entropy(multi_sine, 256)
print(f"  Pure 10 Hz sine:    H_spectral = {H_pure:.4f}")
print(f"  Multi-frequency:    H_spectral = {H_multi:.4f}")
print("  → Multi-frequency has HIGHER spectral entropy (power spread)")

# Exercise 4
print("\n" + "=" * 60)
print("EXERCISE 4: Entropy and Neural States (Discussion)")
print("=" * 60)
print("  Expected patterns:")
print("  • Eyes closed: LOWER spectral entropy (strong alpha)")
print("  • Eyes open: HIGHER spectral entropy (alpha suppression)")
print("  • Cognitive task: Variable (depends on task demands)")
print("  • Deep sleep: LOWER entropy (slow, synchronized waves)")
print("  • REM sleep: HIGHER entropy (desynchronized, dream-like)")
print("  • Awake: Intermediate entropy")

## 15. Summary

### Key Concepts

| Concept | Definition | Key Property |
|---------|------------|--------------|
| **Information** | Reduction of uncertainty | Measured in bits |
| **Entropy** | Average uncertainty of a random variable | $H = -\sum p \log p$ |
| **Maximum Entropy** | When all outcomes equally likely | $H_{max} = \log(n)$ |
| **Binary Entropy** | Entropy for yes/no questions | Max at p = 0.5 |

### Entropy Variants for Signals

| Type | Domain | Measures |
|------|--------|----------|
| **Amplitude Entropy** | Time/Value | Uncertainty in signal values |
| **Spectral Entropy** | Frequency | How spread is power across frequencies |
| **Sample Entropy** | Time patterns | Regularity/complexity of patterns |

### Practical Considerations

1. **Binning choice** affects discrete entropy estimation
2. **Bias correction** (Miller-Madow) helps with small samples
3. **Normalization** allows comparison across different state spaces
4. **Sample entropy** avoids binning issues

### Connection to Connectivity

Entropy is the foundation for:
- **Mutual Information** (D02): Shared entropy between signals
- **Transfer Entropy** (D03): Directed information flow

### Functions Defined

```python
compute_entropy_discrete(probabilities, base)      # Discrete entropy
compute_entropy_from_counts(counts, base)          # From observations
compute_max_entropy(n_states, base)                # Maximum possible
compute_normalized_entropy(probabilities, base)    # Range 0-1
binary_entropy(p)                                  # H(p) for binary
optimal_n_bins(n_samples, method)                  # Bin selection
compute_entropy_continuous(signal, n_bins, method) # For continuous signals
compute_entropy_miller_madow(signal, n_bins)       # Bias-corrected
compute_spectral_entropy(signal, fs, ...)          # Frequency domain
compute_sample_entropy(signal, m, r)               # Pattern-based
```

## 16. Discussion Questions

1. **Why is entropy measured in bits?**
   - What's the connection to binary encoding?
   - How does this relate to computer science?

2. **Entropy and brain states**
   - Would you expect higher or lower entropy during focused attention vs. mind wandering?
   - How might anesthesia affect brain signal entropy?

3. **Limitations of entropy**
   - What information does entropy NOT capture about a signal?
   - Why might two very different signals have the same entropy?

4. **Choosing entropy measures**
   - When would you prefer amplitude entropy vs. spectral entropy vs. sample entropy?
   - What are the trade-offs of each approach?

5. **Preview: From entropy to connectivity**
   - How might we use entropy to measure "shared information" between two brains?
   - What would it mean if two signals have high mutual entropy?

---

## Next Steps

In the next notebook (**D02: Mutual Information**), we'll learn:
- How to quantify **shared information** between signals
- The relationship $I(X;Y) = H(X) + H(Y) - H(X,Y)$
- How mutual information reveals **statistical dependencies**
- Application to inter-brain connectivity in hyperscanning

---

*Notebook completed! Entropy is the foundation of information theory — now you're ready to explore connectivity measures based on shared information.*